In [15]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'sklearn.tree.DecisionTreeClassifier'

In [38]:
class DosAttackDetector:
    def __init__(self, normal_dataset_filepath, dos_dataset_filepath,
                 window_size=25, decision_sensitivity=0.5):
        # 正常数据集和Dos攻击文件路径
        self.normal_dataset_filepath = normal_dataset_filepath
        self.dos_dataset_filepath = dos_dataset_filepath
        # 窗口长度
        self.window_size = window_size
        # 决策敏感度
        self.decision_sensitivity = decision_sensitivity
        ##### 其他计算精准度指标的过程中需要用到的中间数据####
        # 正常数据集各时间窗口的 ID 列表[]
        self.normal_window_ids = []
        # 正常数据集各时间窗口的信息熵列表[]
        self.normal_window_entropy = []
        # 正常数据集各时间窗口对应的时间戳列表
        self.normal_window_timestamps = []
        # 正常数据集计算出来的平均信息熵
        self.normal_window_entropy_mean = 0
        # 正常数据集计算出来的信息熵标准差
        self.normal_window_entropy_std = 0
        # 测试数据集各时间窗口的 ID 列表[]
        self.dos_window_ids = []
        # 与测试数据集各时间窗口的 ID 列表[]相对应的信息熵列表[]
        self.dos_window_entropy = []
        # 与测试数据集各时间窗口的 ID 列表[]相对应的时间戳列表[]
        self.dos_window_timestamps = []
        # 与测试数据集各时间窗口的 ID 列表[]相对应的真实标签列表[]
        self.dos_window_attack_true_labels = []
        # 与测试数据集各时间窗口的 ID 列表[]相对应的算法判定标签列表[]
        self.dos_window_attack_Ag_labelss = []
        # 要输出的指标：预测准确率
        self.precision = -1
        # 要输出的指标：假阳性率
        self.fp_rate = -1
        # 所有攻击检测出来的攻击开始时间和攻击检测时间列表
        self.dos_start_timestamp = []
        self.dos_detected_timestamp = []
        # 要输出的指标：响应时间
        self.response_time = -1
    
    # 读取正常数据集文件和dos数据集文件，返回ID和TimeStamp列
    def file_reader(self, file_path):
        res_pd = pd.read_csv(file_path, delimiter=r'\t')
        timestamp = list(res_pd['TimeStamp'])
        IDS = list(res_pd['ID'])
        return IDS,timestamp

    # 传入ID和TimeStamp列和时间窗长度，输出窗口ids列表和窗口时间戳列表
    def generate_windows_ids_timestamp(self, ids, timestamps):
        res_ids_list = []
        res_timestamps_list = []
        for index in range(0, len(ids)-self.window_size):
            res_ids_list.append(ids[index:index+self.window_size])
            res_timestamps_list.append((timestamps[index+self.window_size-1]))
        return res_ids_list, res_timestamps_list
    
    # 填充测试数据各时间窗口的真实标签
    def genereate_window_label(self, ids_list):
        res_label_list = []
        for window_ids in ids_list:
            if 0 in window_ids:
                res_label_list.append(1)
                continue
            res_label_list.append(0)
        return res_label_list
    
    # 填充真实数据集normal_window_ids，normal_window_timestamps
    # 填充 Dos数据集dos_window_ids，dos_window_timestamps，dos_window_attack_true_labels
    def detector_init(self):
        # 真实数据集
        ids,timestamp = self.file_reader(self.normal_dataset_filepath)
        ids_list,timestamps_list = self.generate_windows_ids_timestamp(ids, timestamp)
        self.normal_window_ids = ids_list
        self.normal_window_timestamps = timestamps_list
        # 测试数据集
        ids,timestamp = self.file_reader(self.dos_dataset_filepath)
        ids_list,timestamps_list = self.generate_windows_ids_timestamp(ids, timestamp)
        self.dos_window_ids = ids_list
        self.dos_window_timestamps = timestamps_list
        # 填充测试数据各时间窗口的真实标签
        self.dos_window_attack_true_labels = self.genereate_window_label(self.dos_window_ids)
        
    # 输入窗口ids列表,计算各窗口的信息熵
    def calculate_entropy(self, ids_list):
        var_array,count_array =  np.unique(ids_list,return_counts=True)
        total_count = len(ids_list)
        pcount_array = count_array/total_count
        return -np.sum(pcount_array*np.log(pcount_array))
    
    # 输入一个窗口的信息熵，判断这个窗口是否属于Dos攻击块,是返回1，否则返回0
    def __judge_if_dos(self, window_entropy):
        if window_entropy <= self.normal_window_entropy_mean - self.decision_sensitivity*self.normal_window_entropy_std:
            return 1
        if window_entropy <= self.normal_window_entropy_mean + self.decision_sensitivity*self.normal_window_entropy_std:
            return 1
        return 0
    
    # 计算正常数据集窗口和测试数据集窗口信息熵,正常数据集窗口信息熵均值和标准差
    def __calculate_window_entropy(self):
        # 正常数据集窗口
        self.normal_window_entropy = [self.calculate_entropy(self.normal_window_ids[index]) 
                                      for index in range(len(self.normal_window_ids))]
        self.normal_window_entropy_mean = np.mean(self.normal_window_entropy)
        self.normal_window_entropy_std = np.std(self.normal_window_entropy)
        
        # 测试数据集窗口
        self.dos_window_entropy = [self.calculate_entropy(self.dos_window_ids[index]) 
                                      for index in range(len(self.dos_window_ids))]
        
    # 判断测试数据集各窗口是否属于Dos攻击块
    def __judge_dos_window_label(self):
        self.dos_window_attack_Ag_labels = [self.__judge_if_dos(self.dos_window_entropy[index]) 
                                            for index in range(len(self.dos_window_entropy))]
    
    # 计算指标：预测准确率 以及 假阳性率
    def __calculate_precision_fp_rate(self):
        
        pass
    
    # DosAttackDetector运行主程，返回三个指标：预测准确率、假阳性率以及平均响应时间
    def dos_engine(self):
        # 初始化
        self.detector_init()
        # 计算正常数据集窗口和测试数据集窗口信息熵,正常数据集窗口信息熵均值和标准差
        self.__calculate_window_entropy()
        # 判断测试数据集各窗口是否属于Dos攻击块
        self.__judge_dos_window_label()
        # 计算指标：预测准确率 以及 假阳性率
        pass
    
        

In [39]:
# 文件路径
normal_dataset_filepath = r'E:\Other Program\PersonalProfit\Ddos Detector\DataSet\DoS_attack_dataset_no_zero.csv'
dos_dataset_filepath = r'E:\Other Program\PersonalProfit\Ddos Detector\DataSet\Add_DoS_attack_dataset1.csv'

In [40]:
dos_detector = DosAttackDetector(normal_dataset_filepath=normal_dataset_filepath,
                                 dos_dataset_filepath=dos_dataset_filepath)
dos_detector.detector_init()

A:\Anaconda\lib\site-packages\ipykernel_launcher.py:44: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [ ]:
# test = [1,2,2,1,3,4,4,4,5,6,6]
#test = [1,2,2,1]
test = [1,1,2,46,76,4,3,2,2,6,8,8]
dos_detector.calculate_entropy(test)

[0.16666667 0.25       0.08333333 0.08333333 0.08333333 0.16666667
 0.08333333 0.08333333]
[-1.79175947 -1.38629436 -2.48490665 -2.48490665 -2.48490665 -1.79175947
 -2.48490665 -2.48490665]


1.9792045174343245

0.6931471805599453